In [ ]:
%pip install statsmodels
%pip install holidays

In [7]:
import warnings
warnings.filterwarnings('ignore')

import sys

# Bibliteca para Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Biblioteca para Manipulação de Dados
import pandas as pd
import numpy as np

# Biblioteca para Análise e modelagem de séries temporais
from statsmodels.tsa.seasonal import seasonal_decompose

# Imports para formatação dos gráficos
import matplotlib.cbook
import matplotlib as m
m.rcParams['axes.labelsize'] = 14
m.rcParams['xtick.labelsize'] = 12
m.rcParams['ytick.labelsize'] = 12
m.rcParams['text.color'] = 'k'

# Imports para criação e validação dos modelos temporais
from statsmodels.tsa.arima_model import ARIMA
import sklearn
from sklearn.metrics import mean_squared_error 
import itertools

# Import para Padronização dos dados
from sklearn.preprocessing import StandardScaler

# Import utilizado para Feature Selection
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV

# Import utilizado para análise de MultiColinearidade dos Dados
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Import para Análise de Estacionaridade nos Modelos de Séries Temporais
from statsmodels.tsa.stattools import adfuller

# Import para obter os feriados usados no Modelo SARIMAX Exógeno
import holidays

# Import Modelo de Séries Temporais Multivariado
from statsmodels.tsa.vector_ar.var_model import VAR

# Imports para o modelo Gradient Boosting
from sklearn.ensemble import GradientBoostingRegressor

# Import para aplicar Cross Validation
from sklearn.model_selection import cross_val_score

# Import para definir os KFold do CV
from sklearn.model_selection import RepeatedKFold

# Import para Otimização de HiperParametros
from sklearn.model_selection import GridSearchCV

# Import para o modelo de Regressão Linear
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
import statsmodels.tsa.api as smt
import statsmodels.stats as sms

from matplotlib.pylab import rcParams 
rcParams['figure.figsize'] = 20,10
matplotlib.style.use('ggplot')

%matplotlib inline

In [8]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))